# General imports and functions

In [21]:
from pathlib import Path
import numpy as np
from collections import defaultdict
import re
from functools import lru_cache
import itertools

DATA_ROOT = Path("data")

# Day 1

In [44]:
file = DATA_ROOT / 'day_1' / 'full.txt'
contents = np.genfromtxt(file, dtype=np.int64)

## Part 1

In [31]:
contents[:, 0].sort()
contents[:, 1].sort()
np.abs(np.diff(contents, axis=1)).sum().item()

1941353

## Part 2

In [45]:
numbers, counts = np.unique(contents[:, 1], return_counts=True)
occurences = dict(zip(numbers, counts))
s = np.sum(contents[:, 0] * np.vectorize(lambda x: occurences.get(x, 0))(contents[:, 0]))
s.item()

22539317

# Day 2

In [80]:
file = DATA_ROOT / 'day_2' / 'full.txt'
contents = [[int(x) for x in l.split()] for l in open(file, 'r').readlines()]

## Part 1

In [66]:
s = 0
for report in contents:
    report = np.array(report)
    ascending = np.all(report[:-1] <= report[1:])
    descending = np.all(report[:-1] >= report[1:])
    differences = np.abs(np.diff(report))
    level_changes = (differences.min() >= 1) & (differences.max() <= 3)
    s += (ascending | descending) & level_changes
s.item()

332

## Part 2

In [81]:
def check_report(report):
    ascending = np.all(report[:-1] <= report[1:])
    descending = np.all(report[:-1] >= report[1:])
    differences = np.abs(np.diff(report))
    level_changes = (differences.min() >= 1) & (differences.max() <= 3)
    return (ascending | descending) & level_changes

s = 0
for report in contents:
    report = np.array(report)
    if check_report(report):
        s += 1
    else:
        for i in range(len(report)):
            if check_report(np.concatenate((report[:i], report[i + 1:]))):
                s += 1
                break
s

398

# Day 3

In [109]:
file = DATA_ROOT / 'day_3' / 'full.txt'
contents = ''.join(open(file, 'r').readlines())

## Part 1

In [99]:
sum((int(s.split(',')[0][4:]) * int(s.split(',')[1][:-1]) for s in re.findall(r'mul\(\d{1,3},\d{1,3}\)', contents)))

161

## Part 2

In [110]:
matches = re.findall(r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)', contents)
enabled = True
s = 0
for match in matches:
    if match == 'do()':
        enabled = True
    elif match == 'don\'t()':
        enabled = False
    elif enabled:
        s += int(match.split(',')[0][4:]) * int(match.split(',')[1][:-1])
s

76911921

# Day 4

In [205]:
def check_word(arr, row, col, word='XMAS'):
    nrows, ncols = arr.shape
    word_size = len(word)
    word = np.array(list(word))
    word_backwards = word[::-1]

    count = 0
    if col <= nrows - word_size:
        hor_subword = arr[row, col:col + word_size]
        if  np.array_equal(hor_subword, word) or np.array_equal(hor_subword, word_backwards):
            count += 1
        
    if row <= ncols - word_size:
        vert_subword = arr[row:row + word_size, col]
        if np.array_equal(vert_subword, word) or np.array_equal(vert_subword, word_backwards):
            count += 1

    if row <= nrows - word_size and col <= ncols - word_size:
        diag_subword = np.diagonal(arr[row: row + word_size, col: col + word_size])
        if np.array_equal(diag_subword, word) or np.array_equal(diag_subword, word_backwards):
            count += 1

    if row >= word_size - 1 and col <= ncols - word_size:
        diag_subword = np.diagonal(np.fliplr(arr[row - word_size + 1: row + 1, col: col + word_size]))
        if np.array_equal(diag_subword, word) or np.array_equal(diag_subword, word_backwards):
            count += 1

    return count

def check_cross(arr, row, col):
    if arr[row, col] != 'A':
        return False
    tl = arr[row - 1, col - 1]
    tr = arr[row - 1, col + 1]
    bl = arr[row + 1, col - 1]
    br = arr[row + 1, col + 1]
    return  ((tl == 'M' and br == 'S') or (tl == 'S' and br == 'M')) \
        and ((bl == 'M' and tr == 'S') or (bl == 'S' and tr == 'M'))

file = DATA_ROOT / 'day_4' / 'full.txt'
contents = np.genfromtxt(file, dtype=str, delimiter=1)

## Part 1

In [206]:
nrows, ncols = contents.shape
np.sum([[check_word(contents, row, col) for col in range(ncols)] for row in range(nrows)]).item()

2578

## Part 2

In [207]:
nrows, ncols = contents.shape
np.sum([[check_cross(contents, row, col) for col in range(1, ncols - 1)] for row in range(1, nrows - 1)]).item()

1972

# Day 5

In [30]:
file = DATA_ROOT / 'day_5' / 'full.txt'
contents = open(file, 'r').read()

rules, printing_orders = contents.split('\n\n')
before_dict = defaultdict(set)
for rule in rules.split():
    before, after = map(int, rule.split('|'))
    before_dict[after].add(before)
printing_orders = [list(map(int, order.split(','))) for order in printing_orders.split()]

## Part 1

In [32]:
def is_valid_order(order):
    for i, current_page in enumerate(order[:-1]):
        if any(next_page in before_dict[current_page] for next_page in order[i + 1:]):
            return False
    return True

sum(order[len(order) // 2] for order in [order for order in printing_orders if is_valid_order(order)])

5639

## Part 2

In [37]:
changed_orders = []
for order in printing_orders:
    order = order[:]
    changed = False
    for i in range(len(order) - 1):
        current_page = order[i]
        for j in range(i + 1, len(order)):
            next_page = order[j]
            if next_page in before_dict[current_page]:
                order[i] = next_page
                order[j] = current_page
                current_page = next_page
                changed = True
    if changed:
        changed_orders.append(order)
sum([order[len(order) // 2] for order in changed_orders])

5273

# Day 6

In [71]:
rot = np.array([[0, 1], [-1, 0]])


def get_config(file):
    grid = {}
    guard_pos = None
    guard_orientation = None
    for row, line in enumerate(open(file, 'r').readlines()):
        for col, cell in enumerate(line):
            grid[(row, col)] = '.'
            if cell == '#':
                grid[(row, col)] = '#'
            elif cell == '^':
                guard_pos = np.array([row, col])
                guard_orientation = np.array([-1, 0])
            elif cell == '>':
                guard_pos = np.array([row, col])
                guard_orientation = np.array([0, 1])
            elif cell == 'v':
                guard_pos = np.array([row, col])
                guard_orientation = np.array([1, 0])
            elif cell == '<':
                guard_pos = np.array([row, col])
                guard_orientation = np.array([0, -1])
    return grid, guard_pos, guard_orientation


def get_visited_squares(grid, guard_pos, guard_orientation):
    visited_squares = set([tuple(guard_pos.tolist())])
    next_pos = guard_pos + guard_orientation
    next_pos_tuple = tuple(next_pos.tolist())
    while next_pos_tuple in grid:
        if grid[next_pos_tuple] == '#':
            guard_orientation = rot @ guard_orientation
        else:
            visited_squares.add(next_pos_tuple)
            guard_pos = next_pos
        next_pos = guard_pos + guard_orientation
        next_pos_tuple = tuple(next_pos.tolist())
    return visited_squares


file = DATA_ROOT / 'day_6' / 'full.txt'

## Part 1

In [72]:
visited_squares = get_visited_squares(*get_config(file))
len(visited_squares)

4696

## Part 2

In [73]:
def check_grid(grid, guard_pos, guard_orientation):
    guard_configs = set([tuple(guard_pos.tolist() + guard_orientation.tolist())])
    next_pos = guard_pos + guard_orientation
    next_pos_tuple = tuple(next_pos.tolist())
    next_guard_tuple = tuple(next_pos.tolist() + guard_orientation.tolist())
    while next_pos_tuple in grid and not next_guard_tuple in guard_configs:
        if grid[next_pos_tuple] == '#':
            guard_orientation = rot @ guard_orientation
        else:
            guard_configs.add(next_guard_tuple)
            guard_pos = next_pos
        next_pos = guard_pos + guard_orientation
        next_pos_tuple = tuple(next_pos.tolist())
        next_guard_tuple = tuple(next_pos.tolist() + guard_orientation.tolist())
    return next_guard_tuple in guard_configs


grid, guard_pos, guard_orientation = get_config(file)
visited_squares = get_visited_squares(grid, guard_pos, guard_orientation)
visited_squares.remove(tuple(guard_pos.tolist()))

suitable_locations = 0
for potential_location in visited_squares:
    modified_grid = dict(grid)
    modified_grid[potential_location] = '#'
    if check_grid(modified_grid, guard_pos, guard_orientation):
        suitable_locations += 1
suitable_locations

1443

# Day 7

In [ ]:
file = DATA_ROOT / 'day_7' / 'full.txt'
calibration_list = [[int(calibration_value)] + list(map(int, numbers.split())) for calibration_value, numbers in map(lambda x: x.split(': '), open(file, 'r').readlines())]

## Part 1

In [ ]:
@lru_cache(maxsize=int(2**16))
def check_calibration(calibration_value, numbers, current_value):
    if not numbers:
        return current_value == calibration_value
    if current_value > calibration_value:
        return False
    
    return check_calibration(calibration_value, numbers[1:], current_value + numbers[0]) \
        or check_calibration(calibration_value, numbers[1:], current_value * numbers[0])

sum([l[0] for l in calibration_list if check_calibration(l[0], tuple(l[2:]), l[1])])

12839601725877


## Part 2

In [ ]:
@lru_cache(maxsize=int(2**16))
def check_calibration(calibration_value, numbers, current_value):
    if not numbers:
        return current_value == calibration_value
    if current_value > calibration_value:
        return False
    
    return check_calibration(calibration_value, numbers[1:], current_value + numbers[0]) \
        or check_calibration(calibration_value, numbers[1:], current_value * numbers[0]) \
        or check_calibration(calibration_value, numbers[1:], int(str(current_value) + str(numbers[0])))

sum([l[0] for l in calibration_list if check_calibration(l[0], tuple(l[2:]), l[1])])

149956401519484


# Day 8

In [48]:
file = DATA_ROOT / 'day_8' / 'test.txt'

lines = open(file, 'r').readlines()
grid = {}
nodes = defaultdict(list)
for row, line in enumerate(lines):
    for col, cell in enumerate(line.strip()):
        grid[(row, col)] = cell
        if cell != '.':
            nodes[cell].append(np.array([row, col]))

## Part 1

In [47]:
antinode_locations = set()
for node in nodes:
    for node_0, node_1 in itertools.combinations(nodes[node], 2):
        diff = node_0 - node_1
        antinode_0 = tuple((node_1 + 2 * diff).tolist())
        antinode_1 = tuple((node_0 - 2 * diff).tolist())
        if antinode_0 in grid:
            antinode_locations.add(antinode_0)
        if antinode_1 in grid:
            antinode_locations.add(antinode_1)
len(antinode_locations)

293

## Part 2

In [ ]:
antinode_locations = set()
for node in nodes:
    for node_0, node_1 in itertools.combinations(nodes[node], 2):
        diff = node_0 - node_1
        i = 0
        while True:
            i += 1
            antinode_0 = tuple((node_1 + i * diff).tolist())
            antinode_1 = tuple((node_0 - i * diff).tolist())
            if antinode_0 in grid:
                antinode_locations.add(antinode_0)
            if antinode_1 in grid:
                antinode_locations.add(antinode_1)
            if antinode_0 not in grid and antinode_1 not in grid:
                break
        
len(antinode_locations)